In [ ]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

# 建立 Ollama 模型
"""
ChatOllama 支援聊天提示和多輪對話，比較適合多輪交互；
而 OllamaLLM 主要用於單輸入單輸出，較簡單的應用。
語法和功能不同，ChatOllama 比較適合現在這種聊天場景。
"""
model = ChatOllama(
    model="gemma3:4b",
    temperature=0.7
)


In [2]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_name}"),
    ]
)

In [5]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
)

In [6]:
# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_name": restaurant_info})

print(result)

好的，讓我來為這家台北市中心的義大利餐廳進行一次專業的評論。

**餐廳名稱：** (請提供餐廳的實際名稱，我會將其納入評論中)

**整體評價：** 這是一家具有一定水準的義大利餐廳，憑藉優雅的環境和親切的服務，在台北市中心佔據了一席之地。然而，其高昂的價格和較長的等待時間，可能會影響部分顧客的用餐體驗。

**優點：**

* **環境優雅：** 餐廳的設計和裝飾，營造出輕鬆愜意的氛圍，非常適合約會或特殊場合。細節的處理，例如燈光、音樂、擺盤等，都展現了餐廳對細節的重視。
* **服務親切：** 服務人員的態度親切專業，能迅速回應顧客的需求，提升用餐的舒適度。
* **食物品質不錯：**  這點很重要，說明餐廳在食材選擇和烹飪技巧方面，有一定水準，能夠提供相對美味的義式料理。 (需要更詳細的描述，例如：醬汁的濃郁度、麵條的口感、食材的新鮮度等)
* **適合約會或特殊場合：**  餐廳的環境氛圍，確實使其成為約會或特殊場合的理想選擇。

**缺點：**

* **價格偏高：** 這是目前餐廳最大的問題。與其他同級義大利餐廳相比，價格明顯偏高，需要仔細評估顧客的預算。
* **等待時間較長：**  等待時間過長，可能會影響顧客的用餐心情。餐廳需要檢討訂位系統、廚房效率等問題，以減少等待時間。
* **份量偏少：** 份量偏少，可能無法滿足食慾，尤其是對於食量較大的人來說。 建議餐廳可以考慮增加部分菜品的份量，或者提供一些搭配菜品的方式。

**建議：**

* **優化菜單價格：**  餐廳可以重新審視菜單價格，使其更具競爭力。
* **提升廚房效率：**  改善廚房的流程和效率，減少烹飪時間。
* **增加菜品份量：**  針對部分菜品，可以考慮增加份量，或提供搭配菜品選項。
* **提升訂位系統：**  優化訂位系統，避免顧客長時間等待。

**總體評分：** (我會給予一個評分，例如：4.5/5)

**（請提供餐廳的實際名稱，我會將其納入評論中）**

**備註：** 評論的深度和細緻程度，會隨著您對餐廳的更多資訊提供而有所提升。 例如，您可以提供菜單內容、特定菜品的口味描述、以及用餐時的具體感受等等。  這能讓我更深入地分析餐廳的優缺點，並提供更具體的建議。

---

為了讓評論更完善，請您提供以下資訊：

*   **餐廳名稱**
*   **菜單上